Importing the Necessary Packages


In [ ]:
#import packages
import sklearn.datasets
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Problem 1

In [ ]:
x1_range = np.arange(0, 1, 0.01)
#create train data
random.seed(0)
x1_pos = []
x2_pos = []
x1_neg = []
x2_neg = []
train_data = []
for i in range(10):
    x1, x2 = random.random(), random.random()
    train_data.append(np.array([x1, x2]))

    if x1 > x2:
        x1_pos.append(x1)
        x2_pos.append(x2)
    else:
        x1_neg.append(x1)
        x2_neg.append(x2)
plt.scatter(x1_pos, x2_pos, label='positive')
plt.scatter(x1_neg, x2_neg, label='negative')
plt.plot(x1_range, x1_range, label='x1-x2=0')
plt.legend()
plt.show()

In [ ]:
x3_pos = []
x4_pos = []
x3_neg = []
x4_neg = []


test_data = []
for i in range(5000):
    x3, x4 = random.random(), random.random()
    test_data.append(np.array([x3, x4]))

Now consider the following loss function for training pair (X, y ¯ ) and
weight vector W¯ :
L = max{0, a − y(W¯ · X¯)},
where the test instances are predicted as ˆy = {W¯ · X¯}. For this problem, W¯ = [w1, w2], X¯ = [x1, x2] and
yˆ = (w1x1 +w2x2). A value of a = 0 corresponds to the perceptron criterion and a value of a = 1 corresponds
to hinge-loss.
1. You need to implement the perceptron algorithm without regularization, train it on the 10 points
above, and test its accuracy on 5000 randomly generated points inside the unit square. Generate the
test points using the same procedure as the training points. You need to have your own implementation
of the perceptron algorithm, using the perceptron criterion loss function. (6)

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def custom_where(cond, x_1, x_2):
    return (cond * x_1) + ((~cond) * x_2)


class Perceptron():
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(num_features, 1,
                                   dtype=torch.float32, device=device)
        self.bias = torch.zeros(1, dtype=torch.float32, device=device)

    def forward(self, x):
        linear = torch.add(torch.mm(x, self.weights), self.bias)
        predictions = custom_where(linear > 0., 1, 0).float()
        return predictions

    def backward(self, x, y):
        predictions = self.forward(x)
        errors = (y - predictions)
        return errors

    def train(self, x, y, epochs):
        for e in range(epochs):

            for i in range(y.size()[0]):
                # use view because backward expects a matrix (i.e., 2D tensor)
                errors = self.backward(x[i].view(1, self.num_features), y[i]).view(-1)
                self.weights += (errors * x[i]).view(self.num_features, 1)
                self.bias += errors

    def evaluate(self, x, y):
        predictions = self.forward(x).view(-1)
        accuracy = torch.sum(predictions == y).float() / y.size()[0]
        return accuracy


In [ ]:
# (Hinge Loss)
class Perceptron():
    def __init__(self, num_features):
        self.num_features = num_features
        self.weights = torch.zeros(num_features, 1,
                                   dtype=torch.float32, device=device)
        self.bias = torch.zeros(1, dtype=torch.float32, device=device)

    def forward(self, x):
        linear = torch.add(torch.mm(x, self.weights), self.bias)
        predictions = custom_where(linear > 0., 1, 0).float()
        return predictions

    def backward(self, x, y):
        predictions = self.forward(x)
        errors = (y - predictions)
        return errors

    def train(self, x, y, epochs):
        for e in range(epochs):

            for i in range(y.size()[0]):
                # use view because backward expects a matrix (i.e., 2D tensor)
                errors = self.backward(x[i].view(1, self.num_features), y[i]).view(-1)

                # Change to hingle loss directly below
                self.weights += -1 - (errors * x[i]).view(self.num_features, 1)
                self.bias += errors

    def evaluate(self, x, y):
        predictions = self.forward(x).view(-1)
        accuracy = torch.sum(predictions == y).float() / y.size()[0]
        return accuracy


1C

I obtained better accuracy with the standard implementation without regularization, because using the hinge loss corresponded to a different range of potential values beginning at -1, as opposed to beginning at 0 which was with the standard implementation.

1D
It would most definitely not change when using the standard implementation, as opposed to the hinge loss. Albeit, even if the values were to change in the smallest degree the change would be less apparent as opposed to if it were hinge loss, simlpy because hinge loss has a wider range of potential values that our predictions could take on.


# Problem 2

Part 1: Does true negative matter for both ROC and PR curve ? Argue why each point on ROC curve
corresponds to a unique point on PR curve ? (5)

True Negative (TP) does not matter for ROC curve because the curve only examines the tradeoff that exists between the True Positive Rate (TPR) and False Positive Rate (FNR) (False Negatives/False Positives).


Similarly, for PR Curve, the Curve is purely interested in false positives and true positives.


Each point on the ROC Curve will correspond in a 1:1 fashion to unique points on the PR Curve when the two curves contain identical confusion matricies, only when recall does not equal 0. This is because when our recall equals 0 then we are unable to compute the false positive rate, thus unable to compute a unique confusion matrix.


Part 2

. Select one OpenML dataset with 2 output classes. Use two binary classifiers (Adaboost and Logistic
regression) and create ROC and PR curves for each of them. You will have two figures: one containing
two ROC and other containing two PR curves. Show the point where an all positive classifier lies in
the ROC and PR curves. An all positive classifier classifies all the samples as positive. (10)

In [ ]:
import sklearn
#import openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import precision_score
from sklearn.linear_model import LogisticRegression
# Import the data set
from scipy.io.arff import loadarff

raw_data = loadarff('phpOJxGL9 (1).arff')
df_data = pd.DataFrame(raw_data[0])
df_data.set_index("V2")
df = pd.DataFrame.from_records(raw_data[0])
df.head()
# Transform Byte String values into int
ordinalEconder = OrdinalEncoder().fit_transform(df[['V2']])
ordinalEconder2 = OrdinalEncoder().fit_transform(df[['Class']])
df["V2_binary"] = pd.DataFrame(ordinalEconder).astype(int)
df["Class_binary"] = pd.DataFrame(ordinalEconder2).astype(int)
# Drop the columns that are unuseable
df.drop(columns = ["V2", "Class"], inplace = True)

In [ ]:
# begin train test split
from sklearn.model_selection import train_test_split
X = df.drop(['Class_binary'], axis = 1)
y = df['Class_binary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train_5 = (y_train >= 0)
y_test_5 = (y_test >= 0)
y_test_5

In [ ]:
# AdaBOOST Classifier

from sklearn.ensemble import AdaBoostClassifier
AdaModel = AdaBoostClassifier(n_estimators=100, learning_rate=1)
model = AdaModel.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_correct = model.decision_function(X.values)
#print("Accuracy:", metrics.accuracy_score(y_, y_pred_correct))
#AdaClassifierModel = metrics.accuracy_score(y, y_pred_correct)


# Logistic Regression Classifier
logreg = LogisticRegression(max_iter = 500)
lr = logreg.fit(X_train, y_train)
lrpredict = lr.predict(X_test)
y_scores = lr.decision_function(X.values)
#print("Accuracy for LR:",metrics.accuracy_score(y_test, lrpredict))
#LRModel = metrics.accuracy_score(y, y_scores)

# Plot LR Model ROC Curve



#roc_curve(y, y_scores)


In [ ]:
# Plotting

# Logistic Regression Classifier
logreg = LogisticRegression(max_iter = 500)

lr = logreg.fit(X_train, y_train)
lrpredict = lr.predict(X_test)

y_scores = lr.decision_function(X.values)

print("Accuracy for LR:",metrics.accuracy_score(y_test, lrpredict))
LRModel = metrics.accuracy_score(y_scores, lrpredict)









from matplotlib.pyplot import figure
import matplotlib.pyplot as plt


figure(figsize=(8, 6), dpi=80)


fpr, tpr, thresholds = metrics.roc_curve(y, y_pred_correct)
plt.plot(fpr, tpr, label= "AdaBoost")


fpr, tpr, thresholds = metrics.roc_curve(y, y_scores)
plt.plot(fpr, tpr, label = "LR")






plt.tick_params(axis='x', labelsize=14)
plt.tick_params(axis='y', labelsize=14)

plt.xlabel("False Positive Rate (Recall)", fontsize = 20)
plt.ylabel("True Positive Rate (Fall-Out)", fontsize = 20)

plt.plot([0,1],[0,1], 'red', linewidth=2, linestyle = "dashed")

plt.legend(loc = 'lower right')


In [ ]:
## AUC Curve:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt

disp = plot_precision_recall_curve(model, X_test, y_test)
disp.ax_.set_title('Precision-Recall curve')

#disp = plot_precision_recall_curve(lr, X_test, lrpredict)
#disp.ax_.set_title('Precision-Recall curve')





# As you can see below, this is the all positive points classifier.



# Problem 3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# Define the dataset
class1 = np.array([[-1, -1], [1, 1], [-3, -3], [4, 4]])
class2 = np.array([[-1, 1], [1, -1], [-5, 2], [4, -8]])

# Combine data and create labels
X = np.vstack((class1, class2))
y = np.array([0, 0, 0, 0, 1, 1, 1, 1])  # 0 for class1, 1 for class2

# Part 1: Plot the dataset and check linear separability
plt.figure(figsize=(10, 8))
plt.scatter(class1[:, 0], class1[:, 1], color='blue', label='Class 1', s=100)
plt.scatter(class2[:, 0], class2[:, 1], color='red', label='Class 2', s=100)

# Annotate points
for i, point in enumerate(class1):
    plt.annotate(f"C1: {tuple(point)}", (point[0], point[1]),
                 xytext=(10, 5), textcoords='offset points')
for i, point in enumerate(class2):
    plt.annotate(f"C2: {tuple(point)}", (point[0], point[1]),
                 xytext=(10, -10), textcoords='offset points')

plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.xlabel('x₁')
plt.ylabel('x₂')
plt.title('Dataset Visualization')
plt.legend()

# Try to fit a linear classifier
clf = LogisticRegression()
clf.fit(X, y)

# Create a mesh grid to visualize the decision boundary
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Get predictions on the mesh grid
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Plot the decision boundary
plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)
plt.savefig('dataset_visualization.png')
plt.close()

# Evaluate the accuracy
accuracy = clf.score(X, y)
print(f"Linear classifier accuracy: {accuracy * 100:.2f}%")

# Test if the model correctly classifies all points
predictions = clf.predict(X)
correctly_classified = (predictions == y).all()
print(f"All points correctly classified: {correctly_classified}")
print(f"Misclassified points: {np.sum(predictions != y)} out of {len(y)}")

# Part 2: Define a new 1-dimensional representation
# Let's try a nonlinear transformation: z = x₁ * x₂ (product of features)
z = X[:, 0] * X[:, 1]

plt.figure(figsize=(10, 4))
plt.scatter(z[y==0], np.zeros_like(z[y==0]), color='blue', label='Class 1', s=100)
plt.scatter(z[y==1], np.zeros_like(z[y==1]), color='red', label='Class 2', s=100)

# Annotate points
for i, point in enumerate(z[y==0]):
    plt.annotate(f"{point:.1f}", (point, 0),
                 xytext=(0, 10), textcoords='offset points', ha='center')
for i, point in enumerate(z[y==1]):
    plt.annotate(f"{point:.1f}", (point, 0),
                 xytext=(0, -20), textcoords='offset points', ha='center')

plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.xlabel('z = x₁ * x₂')
plt.yticks([])
plt.title('1D Representation using z = x₁ * x₂')
plt.legend()
plt.savefig('1d_representation.png')
plt.close()

# Fit a classifier on the 1D representation
z = z.reshape(-1, 1)
clf_1d = LogisticRegression()
clf_1d.fit(z, y)
accuracy_1d = clf_1d.score(z, y)
print(f"\n1D classifier accuracy: {accuracy_1d * 100:.2f}%")

# Find the threshold value that separates the classes
threshold = clf_1d.coef_[0][0] * -1.0 / clf_1d.intercept_[0]
print(f"Separating threshold for z: {threshold:.4f}")

# Part 3: Find the equation of the separating hyperplane in the original space
# For the original 2D space
w = clf.coef_[0]
b = clf.intercept_[0]
print(f"\nOriginal space hyperplane equation: {w[0]:.4f}x₁ + {w[1]:.4f}x₂ + {b[0]:.4f} = 0")

# For the transformed 1D space
w_1d = clf_1d.coef_[0][0]
b_1d = clf_1d.intercept_[0]
print(f"1D space hyperplane equation: {w_1d:.4f}z + {b_1d:.4f} = 0")
print(f"Which translates to: {w_1d:.4f}(x₁*x₂) + {b_1d:.4f} = 0")

# Let's plot the nonlinear decision boundary in the original space
plt.figure(figsize=(10, 8))
plt.scatter(class1[:, 0], class1[:, 1], color='blue', label='Class 1', s=100)
plt.scatter(class2[:, 0], class2[:, 1], color='red', label='Class 2', s=100)

# Create a mesh grid
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                     np.arange(y_min, y_max, 0.01))

# Apply our transformation and predict
zz = xx * yy
grid_z = clf_1d.predict(zz.reshape(-1, 1)).reshape(xx.shape)

# Plot the nonlinear decision boundary
plt.contourf(xx, yy, grid_z, alpha=0.3, cmap=plt.cm.coolwarm)

# Plot the curve where x₁*x₂ = threshold
x1_vals = np.linspace(x_min, x_max, 1000)
x2_vals_upper = threshold / x1_vals
x2_vals_lower = threshold / x1_vals

plt.plot(x1_vals, x2_vals_upper, 'k-', label=f'x₁*x₂ = {threshold:.2f}')

plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.xlabel('x₁')
plt.ylabel('x₂')
plt.title('Nonlinear Decision Boundary in Original Space')
plt.legend()
plt.savefig('nonlinear_boundary.png')
plt.close()

Linear classifier accuracy: 50.00%
All points correctly classified: False
Misclassified points: 4 out of 8

1D classifier accuracy: 100.00%
Separating threshold for z: -19688.9435


IndexError: invalid index to scalar variable.

## Problem 5:

In [ ]:
!pip install utils
!pip install tensorflow==2.7.0.
from tensorflow.keras import backend as K
#2.9.2

# 5a

**Explain vanishing gradients phenomenon using standard normalization with different values of standard
deviation as given in the reference.**

In [ ]:
from tensorflow import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from keras import initializers
from keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten

from utils import (
    compile_model,
    create_mlp_model,
    get_activations,
    grid_axes_it,
)

import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'utils'

In [ ]:
seed = 10

# Number of points to plot
n_train = 1000
n_test = 100
n_classes = 10

# Network params
n_hidden_layers = 5
dim_layer = 100
batch_size = n_train
epochs = 1

# Load and prepare MNIST dataset.
n_train = 60000
n_test = 10000

(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes = len(np.unique(y_test))
data_dim = 28 * 28

x_train = x_train.reshape(60000, 784).astype('float32')[:n_train]
x_test = x_test.reshape(10000, 784).astype('float32')[:n_train]
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

NameError: name 'mnist' is not defined

In [ ]:
def get_activations(model, x, mode=0.0):
    """Extract activations with given model and input vector x."""
    outputs = [layer.output for layer in model.layers]
    activations = K.function([model.input], outputs)
    output_elts = activations(x)
    return output_elts



def exp5a(activation="relu",glorot=False):
    # Run the data through a few MLP models and save the activations from
    # each layer into a Pandas DataFrame.
    rows = []
    sigmas = [0.10, 0.14, 0.28]
    # mode=0.0
    for stddev in sigmas:
        if glorot==False:
            init = initializers.RandomNormal(mean=0.0, stddev=stddev, seed=seed)
        elif glorot==True:
            init = initializers.GlorotNormal()
#         activation = 'relu'

        model = create_mlp_model(
            n_hidden_layers,
            dim_layer,
            (data_dim,),
            n_classes,
            init,
            'zeros',
            activation,
        )
        compile_model(model)
        output_elts = get_activations(model, x_test)
    #     outputs = [layer.output for layer in model.layers]
    #     activations = K.function([model.input], outputs)
    #     output_elts = activations(x_test)
        n_layers = len(model.layers)
        i_output_layer = n_layers - 1

        for i, out in enumerate(output_elts[:-1]):
            if i > 0 and i != i_output_layer:
                for out_i in out.ravel()[::20]:
                    rows.append([i, stddev, out_i])

    df = pd.DataFrame(rows, columns=['Hidden Layer', 'Standard Deviation', 'Output'])

        # Plot previously saved activations from the 5 hidden layers
    # using different initialization schemes.
    fig = plt.figure(figsize=(12, 6))
    axes = grid_axes_it(len(sigmas), 1, fig=fig)
    for sig in sigmas:
        ax = next(axes)
        ddf = df[df['Standard Deviation'] == sig]
        sns.violinplot(x='Hidden Layer', y='Output', data=ddf, ax=ax, scale='count', inner=None)

        ax.set_xlabel('')
        ax.set_ylabel('')

        ax.set_title('Weights Drawn from $N(\mu = 0, \sigma = {%.2f})$' % sig, fontsize=13)

        if sig == sigmas[1]:
            ax.set_ylabel("{} Neuron Outputs".format(activation.capitalize()))
        if sig != sigmas[-1]:
            ax.set_xticklabels(())
        else:
            ax.set_xlabel("Hidden Layer")

    plt.tight_layout()
    plt.show()
    return None

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 25)

## Explain vanishing gradients phenomenon using standard normalization with different values of standard deviation as given in the reference.**bold text**

In [ ]:
exp5a()

NameError: name 'exp5a' is not defined

In [ ]:
## Train the model with tanh activation function

In [ ]:
exp5a("tanh")

NameError: name 'exp5a' is not defined

## Train the model with sigmoid activation function.

In [ ]:
exp5a("sigmoid")

NameError: name 'exp5a' is not defined

## show how Xavier (aka Glorot normal) initialization of weights helps in dealing with this problem.

### tanh activation with glorot normalization

In [ ]:
exp5a("tanh",glorot=True)

NameError: name 'exp5a' is not defined

In [ ]:

### sigmoid activation with glorot normalization

In [ ]:
exp5a("sigmoid",glorot=True)


**bold text**# 5b

The dying ReLU is a kind of vanishing gradient, which refers to a problem when ReLU neurons become
inactive and only output 0 for any input. In the worst case of dying ReLU, ReLU neurons at a certain
layer are all dead, i.e., the entire network dies and is referred as the dying ReLU neural networks in
Lu et al (reference below). A dying ReLU neural network collapses to a constant function. Show this
phenomenon using any one of the three 1-dimensional functions in page 13 of Lu et al. Use a ReLU
network with 10 hidden layers, each of width 2 (hidden units per layer). Use minibatch of 64 and draw
training data uniformly from $[-\sqrt{7},\sqrt{7}]$. Perform 1000 independent training simulations each with
3,000 training points. Out of these 1000 simulations, what fraction resulted in neural network collapse.
Is your answer close to over 90% as was reported in Lu et al.?


$$f_2(x) = x\textrm{sin}(5x)$$

In [ ]:
# Network params
n_hidden_layers = 10
dim_layer = 2
batch_size = 64
input_shape=(1,)
# kernel_initializer = initializers.RandomNormal(mean=0.0,stddev=0.10,seed=seed)
# bias_initializer = 'zeros'
activation = "relu"

def create_mlp5b(n_hidden_layers,
                dim_layer,
                input_shape,
#                 kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.10,seed=seed),
#                 bias_initializer='zeros',
                activation):
    model = Sequential()
    model.add(Dense(dim_layer, input_shape=input_shape)) #, kernel_initializer=kernel_initializer,
#                    bias_initializer=bias_initializer))
    for i in range(n_hidden_layers):
        model.add(Dense(dim_layer, activation=activation)) #, kernel_initializer=kernel_initializer,
#                         bias_initializer=bias_initializer))
    model.add(Dense(1, activation='linear')) #, kernel_initializer=kernel_initializer,
#                     bias_initializer=bias_initializer))
    model.compile(loss="mse",optimizer=keras.optimizers.RMSprop(),metrics=['mse','mae'])
    return model

In [ ]:
n_sims=1000
epochs=1

collapse=[]
yhats=[]
value=np.sqrt(7)
for i in range(n_sims):
    X=np.random.uniform(-value,value,3000)
    Y=np.multiply(X,np.sin(5*X))
#     Y=np.abs(X)
    model=create_mlp5b(n_hidden_layers,dim_layer,input_shape,activation="relu")
    model.fit(X,Y,epochs=10,batch_size=batch_size,verbose=0)
    yhats.append(model.predict(X))
    if len(np.unique(yhats[-1]))==1:
        collapse.append(1)
    else:
        collapse.append(0)

# yhats=np.array(yhats)
# print(yhats[:,:10])
print("With ReLu, the fraction of simulations resulting in collapse is: {}%".format(np.sum(collapse)/n_sims*100))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
94/94 [==============================] - 0s 2ms/step


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


...

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
94/94 [==============================] - 0s 2ms/step


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
94/94 [==============================] - 0s 2ms/step
With ReLu, the fraction of simulations resulting in collapse is: 95.3%


# 5c

Run the 1000 training simulations in part 2 with Leaky ReLU activation and keeping everything else
same. Again calculate the fraction of simulations that resulted in neural network collapse. Did Leaky
ReLU help in preventing dying neurons ?

In [ ]:
collapse=[]
yhats=[]
for i in range(n_sims):
    X=np.random.uniform(-value,value,3000)
    Y=np.multiply(X,np.sin(5*X))
#     Y=np.abs(X)
    model=create_mlp5b(n_hidden_layers,dim_layer,input_shape,activation=keras.layers.LeakyReLU(alpha=0.01))
    model.fit(X,Y,epochs=epochs,batch_size=batch_size,verbose=0)
    yhats.append(model.predict(X))
    if len(np.unique(yhats[-1]))==1:
        collapse.append(1)
    else:
        collapse.append(0)

# yhats=np.array(yhats)
# print(yhats[:,:10])
print("With Leaky ReLU, the fraction of simulations resulting in collapse is: {}%".format(np.sum(collapse)/n_sims*100))

NameError: name 'n_sims' is not defined

6 Batch Normalization, Dropout, MNIST
LeNet-5 Layers

Input - conv2d 16 filters, kernelsize 5
Averagepooling2d
Conv2d 16 filters, kernelsize 5
Averagepooling2d
Dense 120 units
Dense 84 units
Output - dense
6.2
Batch normalization is traditionally used in hidden layers, for input layer standard normalization is used. In standard normalization the mean and standard deviation are calculated using the entire training dataset whereas in batch normalization these statistics are calculated for each mini-batch. Train LeNet-5 with standard normalization of input and batch normalization for hidden layers. What are the learned batch norm parameters for each layer ?

The batch norm params are:

[gamma weights, beta weights, moving_mean(non-trainable), moving_variance(non-trainable)]

so the number of params is  4   ×  the size of the previous layer

In [ ]:
# Load and prepare MNIST dataset.
n_train = 60000
n_test = 10000

(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes = len(np.unique(y_test))
data_dim = 28 * 28

# x_train = x_train.reshape(60000, 784).astype('float32')[:n_train]
# x_test = x_test.reshape(10000, 784).astype('float32')[:n_train]
# x_train /= 255
# x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# pad images bc Lenet5 is 32x32 size images
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

In [ ]:
from tensorflow.keras import layers, models

# standard normalization of input

# stdnorm=layers.experimental.preprocessing.Normalization(axis=-1)
# stdnorm.adapt(x_train)
# x_train=stdnorm(x_train)

mu_pixel=x_train.mean()
sigma_pixel=x_train.std()
x_train=(x_train-mu_pixel)/(sigma_pixel)


# normalize test data
x_test=(x_test-mu_pixel)/(sigma_pixel)

In [ ]:
model = models.Sequential()

# Layer 1 INPUT
model.add(layers.Conv2D(filters=6, kernel_size=5,
                        activation='tanh',
                        input_shape=(32,32,1)))
model.add(layers.AveragePooling2D(2))

# Layer 2
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.AveragePooling2D(2))

# Layer 3
model.add(layers.Dense(120, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Flatten())

# Layer 4
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))

# Layer 5 OUTPUT
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

model.fit(x_train,y_train,epochs = 25)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 69s 34ms/step - accuracy: 0.9147 - loss: 0.2883
Epoch 2/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 53s 28ms/step - accuracy: 0.9696 - loss: 0.0984
Epoch 3/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9761 - loss: 0.0819
Epoch 4/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 81s 21ms/step - accuracy: 0.9790 - loss: 0.0701
Epoch 5/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 22ms/step - accuracy: 0.9816 - loss: 0.0618
Epoch 6/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 45s 24ms/step - accuracy: 0.9827 - loss: 0.0577
Epoch 7/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 89s 28ms/step - accuracy: 0.9843 - loss: 0.0515
Epoch 8/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 73s 23ms/step - accuracy: 0.9856 - loss: 0.0462
Epoch 9/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 79s 22ms/step - accuracy: 0.9858 - loss: 0.0478
Epoch 10/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9859 - loss: 0.0487
Epoch 11/25
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 80s 21ms/step - accuracy: 0.9884 - loss: 0.0395
Epoch 12

### batch norm params

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 10, 10, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 5, 5, 120)      │         2,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 5, 5, 120)      │           480 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3000)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 84)             │       252,084 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 84)             │           336 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 516,414 (1.97 MB)

 Trainable params: 257,986 (1007.76 KB)

 Non-trainable params: 440 (1.72 KB)

 Optimizer params: 257,988 (1007.77 KB)

In [ ]:
print("The batch norm params are:")

for i in [4,5,6,7,10,11,12,13,16,17,18,19]:
    print(model.get_weights()[i])

The batch norm params are:
[1.2022235 1.1744932 1.26037   1.1746271 1.352257  1.1002202 0.936707
 1.1379085 1.3885221 1.2554003 1.321574  1.2502552 1.1868576 1.4409424
 1.3345319 1.0720986]
[-0.16338229 -0.38761866 -0.31437886  0.05569484  0.1101559  -0.63331705
  0.18579583  0.10339542 -0.11606093  0.48124132  0.5307401  -0.05816874
  0.44768432 -0.04314415 -0.10108466  0.53196955]
[ 0.81259525 -0.32424405 -0.45299807  0.24487644  0.7163605  -0.6877745
 -0.5116265  -0.4178951  -0.7159991  -0.5160402  -0.49338502  0.36335725
 -0.40873158  0.85438365  0.61860496  0.683597  ]
[0.15202902 0.43473968 0.47284308 0.5405703  0.1684036  0.26375696
 0.31425178 0.424734   0.2140988  0.45881984 0.4711942  0.49673486
 0.3090501  0.12323137 0.3863639  0.2803751 ]
[0.84262794 0.8469518  0.8979906  0.8517515  0.8508223  0.78517264
 0.72348374 0.8495086  0.40393922 0.7840238  0.7418439  0.8074528
 0.80462694 0.9962864  0.9567651  0.74578947 1.0936348  1.0522282
 0.96986747 0.6105847  0.7056657  0.8229

In [ ]:
**bold text**## 6.3

Next instead of standard normalization use batch normalization for input layer also and train the
network. Plot the distribution of learned batch norm parameters for each layer (including input) using
violin plots. Compare the train/test accuracy and loss for the two cases ? Did batch normalization for
input layer improve performance ?

SyntaxError: invalid syntax (<ipython-input-20-b4a73a5a6e68>, line 1)

In [ ]:
model = models.Sequential()

# Layer 1 INPUT
model.add(layers.Conv2D(filters=6, kernel_size=5,
#                         activation='tanh',
                        input_shape=(32,32,1)))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Activation('tanh'))
model.add(layers.AveragePooling2D(2))

# Layer 2
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.AveragePooling2D(2))

# Layer 3
model.add(layers.Dense(120, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Flatten())

# Layer 4
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))

# Layer 5 OUTPUT
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

model.fit(x_train,y_train,epochs = 25)

In [ ]:
y_hat = model.predict(x_test)
y_hat_labels = np.argmax(y_hat, axis = 1)
y_test_labels=np.argmax(y_test,axis=1)

print("accuracy on batch norm input with batch norm on hidden layers")

accuracy_score(y_test_labels,y_hat_labels)

### batch norm params

In [ ]:
# batch normalization params are in indices [2,3,4,5,8,9,10,11,14,15,16,17,20,21,22,23]

# for i,j in enumerate(model.get_weights()):
#     print(i,len(j))

In [ ]:
print("The batch norm params are:")

for i in [2,3,4,5,8,9,10,11,14,15,16,17,20,21,22,23]:
    print(model.get_weights()[i])

bold text## 6.4

Train the network without batch normalization but this time use dropout. For hidden layers use dropout
probability of 0.5 and for input layer take it to be 0.2 Compare test accuracy using dropout to test
accuracy obtained using batch normalization in part 2 and 3.

In [ ]:
model = models.Sequential()

# Layer 1 INPUT
model.add(layers.Conv2D(filters=6, kernel_size=5,
                        activation='tanh',
                        input_shape=(32,32,1)))
model.add(layers.Dropout(0.2))
model.add(layers.AveragePooling2D(2))

# Layer 2
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.Dropout(0.5))
model.add(layers.AveragePooling2D(2))

# Layer 3
model.add(layers.Dense(120, activation='tanh'))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())

# Layer 4
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dropout(0.5))

# Layer 5 OUTPUT
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

model.fit(x_train,y_train,epochs = 25)

In [ ]:
y_hat = model.predict(x_test)
y_hat_labels = np.argmax(y_hat, axis = 1)
y_test_labels=np.argmax(y_test,axis=1)

print("accuracy with dropout layers")

accuracy_score(y_test_labels,y_hat_labels)

## 6.5

Now train the network using both batch normalization and dropout. How does the performance (test
accuracy) of the network compare with the cases with dropout alone and with batch normalization
alone ?

In [ ]:
model = models.Sequential()

# Layer 1 INPUT
model.add(layers.Conv2D(filters=6, kernel_size=5,
#                         activation='tanh',
                        input_shape=(32,32,1)))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Activation('tanh'))
model.add(layers.Dropout(0.2))
model.add(layers.AveragePooling2D(2))

# Layer 2
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Dropout(0.5))
model.add(layers.AveragePooling2D(2))

# Layer 3
model.add(layers.Dense(120, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())

# Layer 4
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.BatchNormalization(axis=-1))
model.add(layers.Dropout(0.5))

# Layer 5 OUTPUT
model.add(layers.Dense(10, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

model.fit(x_train,y_train,epochs = 25)

In [ ]:
y_hat = model.predict(x_test)
y_hat_labels = np.argmax(y_hat, axis = 1)
y_test_labels=np.argmax(y_test,axis=1)

print("accuracy with both batch norm dropout layers")

accuracy_score(y_test_labels,y_hat_labels)